#TUTORIAL: DEEP FEATURE BASED VISUAL SEARCH
This tutorial give a short implementation for the idea of using visual deep features for searching on a collection of images. The deep feature is deployed by using the last convolutional layer of VGG-16 pretrained model.This feature will be represented for an image and used for further seaching step. To compare two images, we comapre their representation vectors with some metrics such as distance and similarity.

# Step 1 - Preparation
This step setups a connection between the Colab server to the Google Drive server (authenticated by your Google account). By this way, you can synchronize the computational machine with your personal data.

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
# Check GDrive connection
# NOTE THAT: You should change this path based on your real configuration
tutorial_path = '/content/drive/MyDrive/1. Teaching/Information Retrieval/Tutorial/'
%cd '$tutorial_path'

/content/drive/MyDrive/1. Teaching/Information Retrieval/Tutorial


In [40]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
import glob

After connect to the drive, we continue to load pretrained VGG-16 model. You can replace VGG-16 by others new ones such as: Inception V3, ResNet-x, DenseNet-x,...

In [41]:
# Load model
model = VGG16(weights='imagenet', include_top=False)
# Mô hình này đã được train trên tập dữ liệu
# Image Net với hàng triệu ảnh và hàng ngàn lớp đối tượng
#model.summary()

#Step 2 - Extract deep feature
In this step, we use the pretrained VGG-16 model to extract the feature at the last convolutional layer. This feature contains high level feature with concepts.

In [ ]:
def extract_vgg16(img_path, model):
    # Buoc 1: Load anh tu file va resize sang 
    # kich thuoc 224 x 224
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    # Buoc 2: Feed anh vao model de duoc feature
    feature = model.predict(img_data)
    feature_np = np.array(feature)
    feature_flat = feature_np.reshape(-1)
    return feature_flat

Next, we scan all image in the data directory and extract features.

In [42]:
paths = glob.glob(tutorial_path + '/data/image/*.jpg')
feats = []
for i, path in enumerate(paths):
    print('Extracting deep feature of image {}: {}'.format(i,path))
    # Extract features
    feat = extract_vgg16(path, model)
    #norm_feat = feat/np.sum(feat)
    feats.append(feat)


Extracting deep feature of image 0: /content/drive/MyDrive/1. Teaching/Information Retrieval/Tutorial//data/image/beach_and_girl.jpg
Extracting deep feature of image 1: /content/drive/MyDrive/1. Teaching/Information Retrieval/Tutorial//data/image/beach_and_girl2.jpg
Extracting deep feature of image 2: /content/drive/MyDrive/1. Teaching/Information Retrieval/Tutorial//data/image/street_food2.jpg
Extracting deep feature of image 3: /content/drive/MyDrive/1. Teaching/Information Retrieval/Tutorial//data/image/dog.jpg
Extracting deep feature of image 4: /content/drive/MyDrive/1. Teaching/Information Retrieval/Tutorial//data/image/dog1.jpg
Extracting deep feature of image 5: /content/drive/MyDrive/1. Teaching/Information Retrieval/Tutorial//data/image/car.jpg
Extracting deep feature of image 6: /content/drive/MyDrive/1. Teaching/Information Retrieval/Tutorial//data/image/car_dog.jpg
Extracting deep feature of image 7: /content/drive/MyDrive/1. Teaching/Information Retrieval/Tutorial//data/i

Check feature dimension returned from previous step.

In [43]:
feats = np.array(feats)
print(feats.shape)

(13, 25088)


#Step 3 - Feature matching


In [47]:
# Choose an image in the directory
index = 8
# matching using dot product
sims = np.dot(feats, feats[index,:]) # feats[index,:] --> feature query màu đỏ
# feats -> tập các feature màu đen trong slide
# giải thích tại sao L2 ko tốt bằng dot product hoặc cosine

# Print out the result
print('similarity between the query and dataset: ', sims)
rank = np.argsort(sims)
print('Top 3 ranked list: ')
print(paths[rank[-1]])
print(paths[rank[-2]])
print(paths[rank[-3]])

similarity between the query and dataset:  [ 168302.36  250120.81  396407.56  388567.06  312034.78  263690.2
  444044.5   855159.25 3710655.   2369484.2   221001.42  241347.08
  275023.88]
Top 3 ranked list: 
/content/drive/MyDrive/1. Teaching/Information Retrieval/Tutorial//data/image/cho_ben_thanh_01.jpg
/content/drive/MyDrive/1. Teaching/Information Retrieval/Tutorial//data/image/cho_ben_thanh_03.jpg
/content/drive/MyDrive/1. Teaching/Information Retrieval/Tutorial//data/image/ntdb_01.jpg


To visualize the ranked list, we use HTML code. It should be skipped if you dont know web programming.

In [45]:
from IPython import display
import base64

def append_to_html(query_index, query_path, urls, scores):
  """Display a text query and the top result videos and scores."""
  html = ''
  html += '<h2>Input query: <i>{}</i> </h2><div>'.format(query_index)
  data_uri = base64.b64encode(open(query_path, 'rb').read()).decode('utf-8')
  html += '<img src="data:image/png;base64,{0}" height="224">'.format(data_uri)
  html += '<br><h2>Results:</h2><br> <div>'
  html += '<table><tr>'
  for idx, score in enumerate(scores):
    html += '<th>Rank #{}, Score:{:.2f}</th>'.format(idx+1, score)
  html += '</tr><tr>'
  for i, url in enumerate(urls):
    html += '<td>'
    data_uri = base64.b64encode(open(url, 'rb').read()).decode('utf-8')
    html += '<img src="data:image/png;base64,{0}" height="224">'.format(data_uri)
    html += '</td>'
  html += '</tr></table></div></div>'
  return html

In [48]:
urls = []
scores = []
for i in range(len(sims)):
  urls.append(paths[rank[-i-1]])
  scores.append(sims[-i-1])
html = append_to_html('Image: ' + str(index), paths[index], urls, scores)
display.HTML(html)

Output hidden; open in https://colab.research.google.com to view.